In [37]:
import numpy as np
#in case we need to repeat experiment
#np.random.seed(255)

import pandas as pd
pd.options.display.max_rows = 22

import matplotlib.pyplot as plt
plt.style.use('classic')

import seaborn as sns
sns.set()

#sklearn imports
from sklearn.cluster import KMeans
from sklearn import model_selection
#from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from pandas.api.types import CategoricalDtype
import pickle # allows for model to be saved/load to file


#Use print instead of display when run as python script
pyscript = True

#Classifier verborsity where supported
verbose_level=3

#Sample Size (5963272)
#sampleN = 3000000
#SampleN = 5963000
sampleN = 3500000

import time

#datafile
#datafile = 'NCDB_FULL_Removed_All_Missing_Values_Multi_Class.csv'
datafile = 'NCDB_FULL_Removed_All_Missing_Values_Binary_Class_Transformed.csv'

#Model Store
file_lr = 'lr_nov_30.model'
file_lr_l1 = 'lr_l1_nov_30.model'
file_dt = 'dt_nov_30.model'
file_svm = 'svm_nov_30.model'
file_knn = 'knn_nov_30.model'
file_mlp = 'mlp_nov_30.model'
file_kmean = 'kmean_nov_30.model'

#Enable Optimization Algorithms
enable_grid_search = False
svm_c = 1
svm_gamma = 1
feature_all = True
defaultFeatures = ['P_AGE', 'V_YEAR', 'C_HOUR', 'C_YEAR', 'C_MNTH', 'C_CONF', 'C_WDAY', 'C_VEHS', 'P_USER', 'P_SEX']

enable_lr_l1 = True
predict_lr_l1 = True

# Enable Algorithms
enable_lr = True
enable_dt = True
enable_svm = True
enable_knn = True
enable_mlp = True
enable_kmean = True

predict_lr = True
predict_dt = True
predict_svm = True
predict_knn = True
predict_mlp = True


#Multiclass classification, binary if falase
multiclass = False

In [38]:
print("Sample size: {}".format(sampleN))

if multiclass:
    print("Multi-Class Classification: Enabled")
else:
    print("Multi-Class Classification: Disabled")

if enable_grid_search:
    print("Grid Search: Enabled")
else:
    print("Grid Search: Disabled")

if feature_all:
    print("All Features: Enabled")
else:
    print("All Features: Disabled")
    
if enable_kmeans:
    print("K-means: Enabled")
else:
    print("K-means: Disabled")

if enable_lr_l1:
    print("Logistic Regression: Enabled")
else:
    print("Logistic Regression: Disabled")
    
if enable_dt:
    print("Decision Tree: Enabled")
else:
    print("Decision Tree: Disabled")
    
if enable_svm:
    print("Support Vector Machines: Enabled")
else:
    print("Support Vector Machines: Disabled")

if  enable_knn:
    print("KNN: Enabled")
else:
    print("KNN: Disabled")
    
if enable_mlp:
    print("MLP: Enabled")
else:
    print("MLP: Disabled")


Sample size: 3500000
Multi-Class Classification: Disabled
Grid Search: Disabled
All Features: Enabled
K-means: Disabled
Logistic Regression: Enabled
Decision Tree: Enabled
Support Vector Machines: Enabled
KNN: Enabled
MLP: Enabled


In [39]:
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ))

Fri Nov 30 21:35:17 2018


In [40]:
#load data
df = pd.read_csv(datafile, engine = 'python')
print(df.head(2))

   C_YEAR  C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  \
0       1       1       1       1       2      34       2       1       1   
1       1       1       1       1       2      34       2       1       1   

   C_RALN  C_TRAF  V_YEAR  P_SEX  P_AGE  P_PSN  P_USER  P_ISEV  
0       1       1       2      0     33      1       1       2  
1       1       1       2      0     70      1       1       1  


In [41]:
print(df.isnull().sum().sum())

0


In [42]:
print(df[df.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())

0


In [43]:
df_cat = df.astype('category').copy()

In [44]:
totalRows = df_cat.index.size
print("Number of Rows in the dataset: {}".format(totalRows))

Number of Rows in the dataset: 3655334


In [45]:
print(df_cat.columns)

Index(['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG',
       'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_YEAR', 'P_SEX', 'P_AGE',
       'P_PSN', 'P_USER', 'P_ISEV'],
      dtype='object')


In [46]:
#One-Hot-Encoding of categorical
#TBD

## Convert Class Variable to Binary if multi class disabled

In [47]:
## Convert Class Variable to Binary
### Merge Injury and Fatality as a single class
### we will compare the results.
if multiclass:
    #Undersample the majority for the 3 class evaluation
    
    df_class = df_cat.copy()
    
    # subset fatal class
    is_fatal =  df_class['P_ISEV']==3
    is_fatal_count = is_fatal.sum()
    print("Number of Fatal: {}".format(is_fatal_count))
    df_class_fatal = df_class[is_fatal]
    print(df_class_fatal.head(2))
    
    # subset injury class
    is_injury =  df_class['P_ISEV']==2
    is_injury_count = is_injury.sum()
    print("Number of Injury: {}".format(is_injury_count))
    df_class_injury = df_class[is_injury]
    print(df_class_injury.head(2))
    
    # subset non_injury class
    is_safe =  df_class['P_ISEV']==1
    is_safe_count = is_safe.sum()
    print("Number of Non-Injury: {}".format(is_safe_count))
    df_class_safe = df_class[is_safe]
    print(df_class_safe.head(2))
    
    # get the size of fatal datafram
    min_size = df_class_fatal.index.size
    print("Size of Fatal Subset: {}".format(min_size))
    
    # get size of injury
    print("Size of injury Subset: {}".format(df_class_injury.index.size))
    
    # size of non-fatal
    print("Size of non-fatal Subset: {}".format(df_class_safe.index.size))
    
    # randomly sample n number of injury and no injury and append to fatal
    df_class_injury_select = df_class_injury.sample(n=min_size)
    print("Shape of injury sampled dataframe: {}".format(df_class_injury_select.shape))
    df_class_safe_select = df_class_safe.sample(n=min_size)
    print("Shape of nom-injury sampled dataframe: {}".format(df_class_safe_select.shape))
    
    #concat the three dataframes
    df_underSample = pd.concat([df_class_fatal, df_class_injury_select, df_class_safe_select])
    print(df_underSample.shape)
    
    #TBD
    if sampleN < df_underSample.index.size:
        df_sample = df_underSample.sample(n=sampleN)
    else:
        df_sample = df_underSample.sample(n=df_underSample.index.size)
    
    #perform the conversion in two steps to avoid any unwanted side effects
    df_sample['P_ISEV'] = df_sample['P_ISEV'].map({1: 'safe', 2: 'injury', 3:'fatal'})
    df_sample['P_ISEV'] = df_sample['P_ISEV'].map({'safe': '0', 'injury': '1', 'fatal':'2'})
    print((df_sample['P_ISEV']=='0').sum())
    print((df_sample['P_ISEV']=='1').sum())
    print((df_sample['P_ISEV']=='2').sum())
    print(df_sample['P_ISEV'].unique())
else:
    df_class = df_cat.copy()

    #perform the conversion in two steps to avoid any unwanted side effects
    df_class['P_ISEV'] = df_class['P_ISEV'].map({1: 'safe', 2: 'injury', 3:'fatal'})
    df_class['P_ISEV'] = df_class['P_ISEV'].map({'safe': '0', 'injury': '1', 'fatal':'1'})
    print((df_class['P_ISEV']=='0').sum())
    print((df_class['P_ISEV']=='1').sum())
    print(df_class['P_ISEV'].unique())
    
    df_sample = df_class.sample(n=sampleN)

print("Size of dataframe for modeling: {}".format(df_sample.index.size))

1570775
2084559
['1' '0']
Size of dataframe for modeling: 3500000


In [48]:
print(df_sample[df_sample.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())

0


In [49]:
# convert to the correct type
df_sample = df_sample.astype('category')
df_sample['C_YEAR'] = df_sample['C_YEAR'].astype(CategoricalDtype(ordered=True))
df_sample['C_MNTH'] = df_sample['C_MNTH'].astype(CategoricalDtype(ordered=True))
df_sample['C_WDAY'] = df_sample['C_WDAY'].astype(CategoricalDtype(ordered=True))
df_sample['C_HOUR'] = df_sample['C_HOUR'].astype(CategoricalDtype(ordered=True))
df_sample['V_YEAR'] = df_sample['V_YEAR'].astype(CategoricalDtype(ordered=True))
df_sample['P_AGE'] = df_sample['P_AGE'].astype('int')
df_sample['P_ISEV'] = df_sample['P_ISEV'].astype('int')
print(df_sample.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3500000 entries, 1384892 to 3358225
Data columns (total 17 columns):
C_YEAR    category
C_MNTH    category
C_WDAY    category
C_HOUR    category
C_VEHS    category
C_CONF    category
C_RCFG    category
C_WTHR    category
C_RSUR    category
C_RALN    category
C_TRAF    category
V_YEAR    category
P_SEX     category
P_AGE     int32
P_PSN     category
P_USER    category
P_ISEV    int32
dtypes: category(15), int32(2)
memory usage: 103.5 MB
None


## Split Training and Testing for Binary class

In [50]:
#Split between data and class
#Y = df_sample[df_sample.columns[-1]]
#X = df_sample[df_sample.columns[0:df_sample.columns.size -1]]

# split data into X and y
X = df_sample.iloc[0:1000,0:16]
Y = df_sample.iloc[0:1000,-1]

In [51]:
print(Y.unique())

[1 0]


In [52]:
print(X.head(3))

        C_YEAR C_MNTH C_WDAY C_HOUR C_VEHS C_CONF C_RCFG C_WTHR C_RSUR C_RALN  \
1384892      7      1      2      1      4     21      1      1      4      3   
1171035      6      1      4      1      2     21      1      4      3      1   
3257595     16     10      3      3      2     36      2      1      1      1   

        C_TRAF V_YEAR P_SEX  P_AGE P_PSN P_USER  
1384892      7      2     1     52     1      1  
1171035      7      3     0     26     1      1  
3257595      1      3     0     27     1      1  


#### Split Test(70%) and Train (30%) for Bianry class 

In [53]:
#sprint into train and test 70/30
#X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.3, random_state=0)
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.3)

## Clustering based on K-Means Clustering

In [54]:
if enable_kmean:
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    print("K-Means Clustering: Start")
    kmeans = KMeans(n_clusters=3, init='random', n_init=10, max_iter=100, tol=1e-04, verbose= verbose_level)
    print(kmeans)
    
    print("K-Means Clustering: Build")
    ykm = kmeans.fit(X)
    
    if pyscript:
        print(ykm.cluster_centers_)
        print(ykm.labels_)
    else:
        display(ykm.cluster_centers_)
        display(ykm.labels_)
    
    # save model to file
    pickle.dump(ykm, open(file_kmean, "wb"))
    
    print("K-Means Clustering: End")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))


Fri Nov 30 21:37:50 2018
K-Means Clustering: Start
KMeans(algorithm='auto', copy_x=True, init='random', max_iter=100,
    n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=3)
K-Means Clustering: Build
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 272196.1085016302
start iteration
done sorting
end inner loop
Iteration 1, inertia 241472.24011975277
start iteration
done sorting
end inner loop
Iteration 2, inertia 233387.44287938677
start iteration
done sorting
end inner loop
Iteration 3, inertia 229839.87104405923
start iteration
done sorting
end inner loop
Iteration 4, inertia 228176.45754562446
start iteration
done sorting
end inner loop
Iteration 5, inertia 227492.86220597313
start iteration
done sorting
end inner loop
Iteration 6, inertia 227187.17864805082
start iteration
done sorting
end inner loop
Iteration 7, inertia 227040.74769395945
start iteration
done sorting
end inner loop


Iteration 9, inertia 226878.58580358443
start iteration
done sorting
end inner loop
Iteration 10, inertia 226878.58580358443
center shift 0.000000e+00 within tolerance 3.279565e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 385332.66765523405
start iteration
done sorting
end inner loop
Iteration 1, inertia 267993.61693192564
start iteration
done sorting
end inner loop
Iteration 2, inertia 243611.45663889428
start iteration
done sorting
end inner loop
Iteration 3, inertia 239599.33117124793
start iteration
done sorting
end inner loop
Iteration 4, inertia 238379.07763076195
start iteration
done sorting
end inner loop
Iteration 5, inertia 238100.06630247316
start iteration
done sorting
end inner loop
Iteration 6, inertia 237742.7017920321
start iteration
done sorting
end inner loop
Iteration 7, inertia 236983.5591265679
start iteration
done sorting
end inner loop
Iteration 8, inertia 235955.75323689723
start iteration
done sorting
end inner lo

### SVM GridSearch for Optimal Parms

In [55]:
#This operation is computationaly expensive.
if enable_grid_search:
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    param_grid = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}
    grid = GridSearchCV(SVC(), param_grid, verbose=verbose_level)
    print(grid)
    grid.fit(X_train, Y_train)
    print(grid.best_params_)
    svm_c = grid.best_params_.get('C')
    svm_gamma = grid.best_params_.get('gamma')
    print(grid.best_estimator_)
    grid_predictions = grid.predict(X_test)
    cfn_matrix_grid = confusion_matrix(Y_test, grid_predictions)
    print(cfn_matrix_grid)
    print(classification_report(Y_test,grid_predictions))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))

## Logistic Regression Model

In [56]:
if enable_lr:
    print("Logistic Regression: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    lr = LogisticRegression(C=1, random_state=0, solver='saga', multi_class='auto', verbose=verbose_level, n_jobs=10)
    print(lr)
    print("Logistic Regression: Fit")
    lr.fit(X_train, Y_train)
    
    # save model to file
    pickle.dump(lr, open(file_lr, "wb"))
    
    
if predict_lr:
    # load model from file
    loaded_model = pickle.load(open(file_lr, "rb"))
    print("Logistic Regression: Predict")
    y_pred = lr.predict(X_test)

    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    # print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Intercept")
    print(lr.intercept_)

    # print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Coefficients")
    print(lr.coef_)

    print("Logistic Regression: Confusion Matrix")
    cnf_matrix_lg = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg)
    
    print("Logistic Regression: Classification Report")
    print(classification_report(Y_test, y_pred))

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
print("Logistic Regression: End")

Logistic Regression: Start
Fri Nov 30 21:37:50 2018
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start=False)
Logistic Regression: Fit
max_iter reached after 0 seconds
Logistic Regression: Predict
Accuracy of logistic regression classifier on train set: 0.60
Accuracy of logistic regression classifier on test set: 0.61
Logistic Regression: Intercept
[0.08195873]
Logistic Regression: Coefficients
[[ 0.04046685  0.00833698  0.02357705 -0.02748427 -0.20673744 -0.01473736
   0.06470615 -0.00215138  0.0864911   0.20766686  0.04873346 -0.17703569
  -0.399519    0.00602553 -0.07988818  0.18707998]]
Logistic Regression: Confusion Matrix
[[ 55  61]
 [ 56 128]]
Logistic Regression: Classification Report
              precision    recall  f1-score   support

           0       0.50      0.47      0.48 

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    0.0s finished


### Logistic Regression with L1 Regularization

In [57]:
if (enable_lr_l1):
    # with L1 regularization
    print("Logistic Regression with L1 Regularization: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    lr = LogisticRegression(penalty='l1', C=1, solver='saga', multi_class='auto', verbose=verbose_level, n_jobs = 10)
    print(lr)
    print("Logistic Regression with L1 Regularization: Fit")
    lr.fit(X_train, Y_train)
    
    # save model to file
    pickle.dump(lr, open(file_lr_l1, "wb"))

if (predict_lr_l1):
    # load model from file
    loaded_model = pickle.load(open(file_lr_l1, "rb"))
    print("Logistic Regression with L1 Regularization: Predict")
    y_pred = lr.predict(X_test)
    
    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    print("Logistic Regression with L1 Regularization: Confusion Matrix")
    cnf_matrix_lg_l1 = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg_l1)
    
    print(classification_report(Y_test,y_pred))
    print("Logistic Regression with L1 Regularization: Classification Report")

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
print("Logistic Regression with L1 Regularization: End")

Logistic Regression with L1 Regularization: Start
Fri Nov 30 21:37:50 2018
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto', n_jobs=10,
          penalty='l1', random_state=None, solver='saga', tol=0.0001,
          verbose=3, warm_start=False)
Logistic Regression with L1 Regularization: Fit
max_iter reached after 0 seconds


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    0.0s finished


Logistic Regression with L1 Regularization: Predict
Accuracy of logistic regression classifier on train set: 0.60
Accuracy of logistic regression classifier on test set: 0.64
Logistic Regression with L1 Regularization: Confusion Matrix
[[ 61  55]
 [ 53 131]]
              precision    recall  f1-score   support

           0       0.54      0.53      0.53       116
           1       0.70      0.71      0.71       184

   micro avg       0.64      0.64      0.64       300
   macro avg       0.62      0.62      0.62       300
weighted avg       0.64      0.64      0.64       300

Logistic Regression with L1 Regularization: Classification Report
Fri Nov 30 21:37:50 2018
Logistic Regression with L1 Regularization: End


### Decision Tree

In [59]:
if enable_dt:
    print("Decision Tree: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    tree = DecisionTreeClassifier(criterion='entropy',max_depth=50)
    print(tree)
    print("Decision Tree: Fit")
    tree.fit(X_train, Y_train)
    # save model to file
    pickle.dump(tree, open(file_dt, "wb"))

if predict_dt:
    # load model from file
    loaded_model = pickle.load(open(file_dt, "rb"))
    print("Decision Tree: Predict")
    y_pred = tree.predict(X_test)
    print('Accuracy of Decision Tree classifier on train set: {:.2f}'.format(tree.score(X_train, Y_train)))
    print('Accuracy of Decision Tree classifier on test set: {:.2f}'.format(tree.score(X_test, Y_test)))
    
    cnf_matrix_dt = confusion_matrix(Y_test, y_pred)
    print("Decision Tree: Confusion Matrix")
    print(cnf_matrix_dt)
    print("Decision Tree: Classification Report")
    print(classification_report(Y_test,y_pred))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    print("Decision Tree: End")

Decision Tree: Start
Fri Nov 30 21:37:50 2018
DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=50,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
Decision Tree: Fit
Decision Tree: Predict
Accuracy of Decision Tree classifier on train set: 1.00
Accuracy of Decision Tree classifier on test set: 0.59
Decision Tree: Confusion Matrix
[[ 61  55]
 [ 68 116]]
Decision Tree: Classification Report
              precision    recall  f1-score   support

           0       0.47      0.53      0.50       116
           1       0.68      0.63      0.65       184

   micro avg       0.59      0.59      0.59       300
   macro avg       0.58      0.58      0.58       300
weighted avg       0.60      0.59      0.59       300

Fri Nov 30 21:37:50 2018
Decisi

### K-N-N

In [60]:
if enable_knn:
    print("KNN: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski', n_jobs = 10)
    print(knn)
    print("KNN: Fit")
    knn.fit(X_train, Y_train)

    # save model to file
    pickle.dump(knn, open(file_knn, "wb"))

if predict_knn:
    # load model from file
    loaded_model = pickle.load(open(file_knn, "rb"))
    
    print("KNN: Predict")
    y_pred = knn.predict(X_test)
    print('Accuracy of KNN classifier on train set: {:.2f}'.format(knn.score(X_train, Y_train)))
    print('Accuracy of KNN classifier on test set: {:.2f}'.format(knn.score(X_test, Y_test)))
    
    print("KNN: Confusion Matrix")
    cnf_matrix_knn = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_knn)

    print("KNN: Classification Report")
    print(classification_report(Y_test,y_pred))

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))

    print("KNN: End")

KNN: Start
Fri Nov 30 21:37:50 2018
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=10, n_neighbors=5, p=2,
           weights='uniform')
KNN: Fit
KNN: Predict
Accuracy of KNN classifier on train set: 0.71
Accuracy of KNN classifier on test set: 0.53
KNN: Confusion Matrix
[[ 48  68]
 [ 73 111]]
KNN: Classification Report
              precision    recall  f1-score   support

           0       0.40      0.41      0.41       116
           1       0.62      0.60      0.61       184

   micro avg       0.53      0.53      0.53       300
   macro avg       0.51      0.51      0.51       300
weighted avg       0.53      0.53      0.53       300

Fri Nov 30 21:37:50 2018
KNN: End


## ANN - Multilayer Perceptron

In [61]:
if enable_mlp:
    print("Multilayer Preceptron: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    
    #mlpc = MLPClassifier(alpha=1)
    #mlpc = MLPClassifier(hidden_layer_sizes=(12, 12, 12), max_iter=100, verbose=verbose_level)
    mlpc = MLPClassifier(hidden_layer_sizes=(25, 25, 25), max_iter=100, verbose=verbose_level)
    print(mlpc)
    #mlp = multilayer_perceptron(n_hidden =2, activation='logistic', algorithm='sgd', random_state=3)
    print("Multilayer Preceptron: fit")
    mlpc.fit(X_train, Y_train)
    
    # save model to file
    pickle.dump(mlpc, open(file_mlp, "wb"))
    
if predict_mlp:
    
    # load model from file
    loaded_model = pickle.load(open(file_mlp, "rb"))
    print("Multilayer Preceptron: Predict")
    y_pred = mlpc.predict(X_test)

    print('Accuracy of Multilayer Perceptron classifier on train set: {:.2f}'.format(mlpc.score(X_train, Y_train)))
    print('Accuracy of Multilayer Perceptron classifier on test set: {:.2f}'.format(mlpc.score(X_test, Y_test)))
    
    print("Multilayer Preceptron: Confusion Matrix")
    cnf_matrix_mlp = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_mlp)
    
    print("Multilayer Preceptron: Classificiation Report")
    print(classification_report(Y_test,y_pred))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Multilayer Preceptron: End")

Multilayer Preceptron: Start
Fri Nov 30 21:37:50 2018
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(25, 25, 25), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=3, warm_start=False)
Multilayer Preceptron: fit
Iteration 1, loss = 1.36585763
Iteration 2, loss = 0.92206233
Iteration 3, loss = 0.85918506
Iteration 4, loss = 0.79060167
Iteration 5, loss = 0.71862777
Iteration 6, loss = 0.72741696
Iteration 7, loss = 0.70384951
Iteration 8, loss = 0.68834516
Iteration 9, loss = 0.68873291
Iteration 10, loss = 0.67848949
Iteration 11, loss = 0.67199238
Iteration 12, loss = 0.67040980
Iteration 13, loss = 0.66651320
Iteration 14, loss = 0.66520833
Iteration 15, loss = 0.

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


### SVM

In [62]:
if enable_svm:
    print("SVM: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    #svm = SVC(C=1, random_state=0, kernel='sigmoid', verbose=True)
    #svm = SVC(C=1, random_state=0, kernel='linear', verbose=True, cache_size=200)
    #svm = SVC(C=svm_c, gamma=svm_gamma, verbose = verbose_level)
    svm = SVC(C=1, gamma = 'auto', verbose = verbose_level)
    print(svm)
    print("SVM: Fit")
    svm.fit(X_train, Y_train)

    # save model to file
    pickle.dump(svm, open(file_svm, "wb"))
    
if predict_svm:
    # load model from file
    loaded_model = pickle.load(open(file_svm, "rb"))
    print("SVM: Predict")
    y_pred = svm.predict(X_test)
    
    print('Accuracy of SVM classifier on train set: {:.2f}'.format(svm.score(X_train, Y_train)))
    print('Accuracy of SVM classifier on test set: {:.2f}'.format(svm.score(X_test, Y_test)))
    
    print("SVM: Confusion Matrix")
    cnf_matrix_svm = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_svm)
    
    print("SVM: Classfication Report")
    print(classification_report(Y_test,y_pred))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("SVM: End")

SVM: Start
Fri Nov 30 21:37:51 2018
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=3)
SVM: Fit
[LibSVM]SVM: Predict
Accuracy of SVM classifier on train set: 0.98
Accuracy of SVM classifier on test set: 0.56
SVM: Confusion Matrix
[[ 32  84]
 [ 48 136]]
SVM: Classfication Report
              precision    recall  f1-score   support

           0       0.40      0.28      0.33       116
           1       0.62      0.74      0.67       184

   micro avg       0.56      0.56      0.56       300
   macro avg       0.51      0.51      0.50       300
weighted avg       0.53      0.56      0.54       300

Fri Nov 30 21:37:51 2018
SVM: End


In [63]:
t_end =  time.time()
print(time.asctime( time.localtime(t_end) ))

Fri Nov 30 21:37:51 2018
